## Annual Report Data Engineering and Analysis

## Setup

In [ ]:
#--------------------------------------------------------------------------------------------------------#
# import packages and modules
# base packages
import os
import sys
import logging
# from datetime import datetime
import pandas as pd
# external connection packages
import requests
# from boxsdk import Client, CCGAuth
import sqlalchemy as sa
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
# ESRI packages
import arcpy
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor
# email packages
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# set overwrite to true
arcpy.env.overwriteOutput = True
arcpy.env.workspace = "C:\GIS\Scratch.gdb"

# in memory output file path
wk_memory = "memory" + "\\"
# set workspace and sde connections 
working_folder = "C:\GIS"
workspace      = "C:\GIS\Scratch.gdb"

# network path to connection files
filePath = "C:\\GIS\\DB_CONNECT"
# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
# Feature dataset to unversion and register as version
fdata = sdeCollect + "\\sde_collection.SDE.Parcel"
# string to use in updaetSDE function
sdeString  = fdata + "\\sde_collection.SDE."
# local path to stage csvs in
accelaFiles = r"F:/GIS/Acella/Reports"

# connect to bmp SQL dataabase
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql14;DATABASE=tahoebmpsde;UID=sde;PWD=staff"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

# # Box API credentialsn setup with CCGAuth
# auth = CCGAuth(
#   client_id     = "pusxamhqx4urav2lj847darrr1niydzp",
#   client_secret = "tmnxqxp8sSY6i24OPX2bAYFrnIA3cerZ",
#   user          = "21689880902"
# )
# # setup client for BOX connection
# client = Client(auth)

##--------------------------------------------------------------------------------------#
## EMAIL and LOG FILE SETTINGS ##
##--------------------------------------------------------------------------------------#
## LOGGING SETUP
# Configure the logging
log_file_path = os.path.join(working_folder, "Parcel_Tables_to_Features_Log.log")  
# setup basic logging configuration
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    filename=log_file_path,  # Set the log file path
                    filemode='w')
# Create a logger
logger = logging.getLogger(__name__)
# Log start message
# logger.info("Script Started: " + str(datetime.datetime.now()) + "\n")

## EMAIL SETUP
# path to text file
fileToSend = log_file_path
# email parameters
subject = "Parcel Tables to Parcel Features ETL"
sender_email = "infosys@trpa.org"
# password = ''
receiver_email = "GIS@trpa.gov"



#### Reports to add to the Batch Engine
* ReviewCompleteness_30Day_V1
* DetailHistory_Reporting

### Count of Permit by Category
*  Get Accela Permits data summarized by count of Reporting Category 
	 - Delete TMP files
	 - Establish Reporting Category
		- Summary of Record Types
			- Get lookup list for Reporting Category by Recor Type
			
		- Filter out Plan Revision
			- A second lookup of the ID without -01 at the end and lookup list against the other File Number Record Type
			
	- Count of Files by Reporting Category
		- Just for current year


In [ ]:
df = pd.read_csv()

### Tree Permits 
* setup Tree Permit Activity report in Batch Engine workflow
    - filter by year
    - merge with the main Accela permit csv by File Number
    - Tree Total
    - CHECKED in a column is the reason the tree was removed 
    - Tree Total is the count of trees that are approved to be removed
    - Application is APPROVED
- How many trees removed apps by year
- Total trees approved by permit
- Get Tree Total by Reason
- just for current year
- what % of applications have x reason CHECKED
- Created By BBARR are the Fire Districts creating a permit on our behalf
    - to look at the ones we've processed filter out the BBARR ones
        - we just want to report on TRPA actions not the Fire district issued permits

In [ ]:
parcelHistory = 
parcelMaster =

OldAPN = 
CurrentAPN = 

### Banked Development Rights Analysis

* All Banked development rights by type, land capability, location and jurisdiction
* Current Banked development rights by type, land capability, location and jurisdiction
    * group by High Capability, Low Capability, and SEZ
    * By location to town center
    * banked before and after 12/12/12

* Remove things that have been banked in the past year from Existing Development on the ground (TAU, RES, CFA)
- quanity that was removed should be subtracted from existing development
- newly built allocations (comes from transacted data in LTinfo) 
    - this falls apart when we dont have current data from the jurisdictions on allocated development
- opportunity to get data from GIS service....

## IPES Score of 0 = SEZ, IPES 1-725 = Low Capability, and IPES>726 = High Capability

#### "F:\Research and Analysis\Reporting\Annual Reports\2022 Annual Report\2022 Annual Report Data\2022 Transfer Analysis.xlsx"

In [ ]:
dfDRTrans  = pd.read_json("https://laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfDRTrans.RecordType.unique()

In [ ]:
dfDRTrans.RecordType.unique()

In [ ]:
dfDRBank   = pd.read_json("https://laketahoeinfo.org/WebServices/GetBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfDRBank

In [ ]:
dfDRBank.DevelopmentRight.unique()

## Get Permit Data

In [ ]:
from pathlib import Path

import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayer

# Gets data from the TRPA server
def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features
    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    # return data frame
    return all_data


def get_spatial_dataframe(service_url):
    # Connect to the GIS
    gis = GIS()

    # Create a FeatureLayer object from the REST service URL
    feature_layer = FeatureLayer(service_url)

    # Create a Spatially Enabled DataFrame from the Feature Layer
    spatial_df = pd.DataFrame.spatial.from_layer(feature_layer)

    return spatial_df

In [ ]:
tbl = "https://maps.trpa.org/server/rest/services/Permit_Records/MapServer/1"
dfPermit = get_fs_data(tbl)

rest_service_url = "https://maps.trpa.org/server/rest/services/Parcels/FeatureServer/0"
sdfParcel = get_spatial_dataframe(rest_service_url)



In [ ]:
dfPermit.Accela_CAPType_Name.unique()

In [ ]:
# get permti dataframe

# get parcel points data

# left join

# create output of stacked points

# publish permits by points

# dot density 